### Multithreaded Video Stream Inference with DeGirum PySDK
This script demonstrates how to run inference on multiple video streams (video files or webcams) simultaneously using DeGirum PySDK. It utilizes multithreading to independently process each stream with different AI models. Each thread runs inference on a specified video source or webcam and displays the results in a separate window. The script supports configurable inference settings, including cloud or local deployment, model zoo location, and token-based authentication. Simply update the configurations list to add or modify streams.

In [ ]:
import threading
import degirum as dg
import degirum_tools

# choose inference host address
inference_host_address = "@cloud"
# inference_host_address = "@local"

# choose zoo_url
zoo_url = "degirum/models_hailort"
# zoo_url = "../models"

# set token
token = degirum_tools.get_token()
# token = '' # leave empty for local inference

# Define the configurations for video file and webcam
configurations = [
    {
        "model_name": "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1",
        "source": "../assets/Traffic.mp4",  # Video file
        "display_name": "Traffic Camera"
    },
    {
        "model_name": "yolov8n_relu6_face--640x640_quant_hailort_hailo8_1",
        "source": 1,  # Webcam index
        "display_name": "Webcam Feed"
    }
]

# Function to run inference on a video stream (video file or webcam)
def run_inference(model_name, source, inference_host_address, zoo_url, token, display_name):
    # Load AI model
    model = dg.load_model(
        model_name=model_name,
        inference_host_address=inference_host_address,
        zoo_url=zoo_url,
        token=token
    )

    with degirum_tools.Display(display_name) as output_display:
        for inference_result in degirum_tools.predict_stream(model, source):
            output_display.show(inference_result)
    print(f"Stream '{display_name}' has finished.")


# Create and start threads
threads = []
for config in configurations:
    thread = threading.Thread(
        target=run_inference,
        args=(
            config["model_name"],
            config["source"],
            inference_host_address,
            zoo_url,
            token,
            config["display_name"]
        )
    )
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

print("All streams have been processed.")